In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from requests import get
import time
import pandas as pd
from random import randint
import datetime

In [2]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
import re

In [43]:
# import json as df
df = pd.read_json ('cs4300sp2020-jjl296-ec629-cc2459-tb444-ak778/app/irsystem/data/final_data1.json', convert_dates=False)

In [58]:
# import json as df
df = pd.read_json ('cs4300sp2020-jjl296-ec629-cc2459-tb444-ak778/data/total_data_2.json', convert_dates=False)

In [45]:
df["transcript"][0]

'16-year-old TJ Kim may not have his driver’s license yet, but he has been flying planes all over Virginia in order to deliver medical supplies to rural hospitals fighting the novel coronavirus.\n\nThe teen, who is a sophomore at Landon School in Bethesda, Maryland, was inspired to embark on his delivery missions after his classes were canceled as a result of COVID-19.\n\nWithout school or lacrosse to occupy his time, he turned his focus on his flying lessons.\n\nRELATED: Man Uses His Savings to Fill Up Gas Tanks For Dozens of Nurses Heading to Nearby Hospital\n\nAccording to The Associated Press, Kim and his family then launched their Operation SOS (Supplies Over Skies) project as a means of keeping busy during the shutdowns while simultaneously serving others in need by delivering supplies to rural hospitals.\n\nWhen Kim first asked his flight instructor, Dave Powell, to use their flying lessons as delivery runs for Virginia hospitals, the pilot was awestruck.\n\nLOOK: College Studen

In [46]:
def tokenize(text, stem_words = True):
    """Returns a tuple containing 
        a list of words that make up the text
        and the number of total words (not including stop words).
    
    Note: for simplicity, lowercase everything.
    Requirement: Use Regex to satisfy this function
    
    Arguments
    =========
    text: String
    stem_words: boolean
    
    Returns
    =======
    Tuple of a list and number 
    
    """
    stop_words = stopwords.words('english')
    stemmer = PorterStemmer()
    regex = r"([a-zA-Z]+)"
    tokens = re.findall(regex, text.lower())
    
    numwords = 0
    words = []
    for t in tokens:
        if not t in stop_words:
            numwords += 1
            if (stem_words):
                words.append(stemmer.stem(t))
            else:
                words.append(t)
    return (words, numwords)

In [47]:
tokenize(df["transcript"][0])

(['year',
  'old',
  'tj',
  'kim',
  'may',
  'driver',
  'licens',
  'yet',
  'fli',
  'plane',
  'virginia',
  'order',
  'deliv',
  'medic',
  'suppli',
  'rural',
  'hospit',
  'fight',
  'novel',
  'coronaviru',
  'teen',
  'sophomor',
  'landon',
  'school',
  'bethesda',
  'maryland',
  'inspir',
  'embark',
  'deliveri',
  'mission',
  'class',
  'cancel',
  'result',
  'covid',
  'without',
  'school',
  'lacross',
  'occupi',
  'time',
  'turn',
  'focu',
  'fli',
  'lesson',
  'relat',
  'man',
  'use',
  'save',
  'fill',
  'ga',
  'tank',
  'dozen',
  'nurs',
  'head',
  'nearbi',
  'hospit',
  'accord',
  'associ',
  'press',
  'kim',
  'famili',
  'launch',
  'oper',
  'so',
  'suppli',
  'sky',
  'project',
  'mean',
  'keep',
  'busi',
  'shutdown',
  'simultan',
  'serv',
  'other',
  'need',
  'deliv',
  'suppli',
  'rural',
  'hospit',
  'kim',
  'first',
  'ask',
  'flight',
  'instructor',
  'dave',
  'powel',
  'use',
  'fli',
  'lesson',
  'deliveri',
  'run',


In [48]:
def make_countsdict(tokens):
    worddict = {}
    for word in tokens:
        if word in worddict:
            worddict[word] += 1
        else:
            worddict[word] = 1
    return worddict

In [53]:
def make_keywords(row, title_scale = 0.3):
    tokenized = tokenize(row["transcript"])
    words = tokenized[0]
    numwords = tokenized[1]
    worddict = make_countsdict(words)
    # divide by numwords
    counts = [(word,count) for word,count in worddict.items()]
    freq = [(word, count/numwords) for word, count in counts]

    # tokenize title
    tokenized_title = tokenize(row["title"])
    titledict = make_countsdict(tokenized_title[0])
    title_freq = [(word, count/tokenized_title[1]) for word, count in titledict.items()]

    # combine
    title_scaled = [(word, count*title_scale) for word, count in title_freq]

    # get top 20
    total = freq + title_scaled
    sortedtotal = sorted(total, key=lambda x: x[1], reverse=True) 
    top_20 = {}
    top = 0
    for word,freq in sortedtotal:
        if word not in top_20:
            top_20[word] = freq
            top += 1
            if top == 20: 
                break
    return top_20
    print(top_20)
    row["keywords"] = top_20

In [13]:
for index, row in df.iterrows():
    print(index)
    make_keywords(row)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069


In [64]:
df["keywords"] = ""

In [68]:
df.head()

,title,url,date,transcript,source,keywords
0,16-Year-Old Has Been Using His Flying Lessons ...,https://www.goodnewsnetwork.org/teen-flies-sup...,4/9/20,16-year-old TJ Kim may not have his driver’s l...,goodnewsnetwork.org,"{'hospit': 0.034482758620689655, 'year': 0.024..."
1,Tyler Perry Picks Up the Tab for All Groceries...,https://www.goodnewsnetwork.org/tyler-perry-pa...,4/9/20,It’s not uncommon for movie mogul Tyler Perry ...,goodnewsnetwork.org,"{'perri': 0.038461538461538464, 'tyler': 0.032..."
2,Taking Hot Baths Every Day is Linked to Lower ...,https://www.goodnewsnetwork.org/daily-hot-bath...,4/8/20,Regular tub bathing is linked to a lower risk ...,goodnewsnetwork.org,"{'bath': 0.04896907216494845, 'risk': 0.028350..."
3,Watch Farmer Use His Tractor to Plow Massive T...,https://www.goodnewsnetwork.org/farmer-plows-t...,4/8/20,A talented farmer has created his own heartwar...,goodnewsnetwork.org,"{'watch': 0.03, 'farmer': 0.03, 'use': 0.03, '..."
4,3 Ways to Deal With the Anxiety of a New Situa...,https://www.goodnewsnetwork.org/dr-brene-brown...,4/8/20,The Lesson: If you have been stressing out ami...,goodnewsnetwork.org,"{'way': 0.03, 'deal': 0.03, 'anxieti': 0.03, '..."


In [60]:
df = df.drop(df.columns[0], axis = 1)

In [83]:
df.to_json('final_data1_keywords.json', orient='records')

In [84]:
with open('final_data1_keywords.json', 'w', encoding='utf-8') as file:
    df.to_json(file, force_ascii=False, orient='records')

_________________________________________________________

# Setting up Scores

In [70]:
keywords = df = pd.read_json ('cs4300sp2020-jjl296-ec629-cc2459-tb444-ak778/data/total_data_2_keywords.json', convert_dates=False)

In [71]:
df.head()

,title,url,date,transcript,source,keywords
0,16-Year-Old Has Been Using His Flying Lessons ...,https://www.goodnewsnetwork.org/teen-flies-sup...,4/9/20,16-year-old TJ Kim may not have his driver’s l...,goodnewsnetwork.org,"{'hospit': 0.0344827586, 'year': 0.025, 'old':..."
1,Tyler Perry Picks Up the Tab for All Groceries...,https://www.goodnewsnetwork.org/tyler-perry-pa...,4/9/20,It’s not uncommon for movie mogul Tyler Perry ...,goodnewsnetwork.org,"{'perri': 0.0384615385, 'tyler': 0.0320512821,..."
2,Taking Hot Baths Every Day is Linked to Lower ...,https://www.goodnewsnetwork.org/daily-hot-bath...,4/8/20,Regular tub bathing is linked to a lower risk ...,goodnewsnetwork.org,"{'bath': 0.048969072200000005, 'risk': 0.02835..."
3,Watch Farmer Use His Tractor to Plow Massive T...,https://www.goodnewsnetwork.org/farmer-plows-t...,4/8/20,A talented farmer has created his own heartwar...,goodnewsnetwork.org,"{'watch': 0.03, 'farmer': 0.03, 'use': 0.03, '..."
4,3 Ways to Deal With the Anxiety of a New Situa...,https://www.goodnewsnetwork.org/dr-brene-brown...,4/8/20,The Lesson: If you have been stressing out ami...,goodnewsnetwork.org,"{'way': 0.03, 'deal': 0.03, 'anxieti': 0.03, '..."


In [83]:
def create_inverted_index(input_data):
    """creates the inverted index
    
    Arguments
    =========
    input_data: dataframe with keywords column (dictionary word: frequency)
    
    Returns
    =======
    {String : list of tuples}
    
    """
    idx = {}
    doc_id = 0
    for index, row in input_data.iterrows():
        for word, freq in row["keywords"].items():
            if idx.get(word) is None:
                idx[word] = []
            idx.get(word).append((index,freq))
    return idx
#     for doc in input_data:
#         for key, val in doc.items():
#             if idx.get(key) is None:
#                 idx[key] = []
#             idx.get(key).append((doc_id, val))
#         doc_id+=1
#     return idx

In [84]:
invidx = create_inverted_index(df)

In [85]:
invidx

{'hospit': [(0, 0.0344827586),
  (13, 0.0344827586),
  (21, 0.0272727273),
  (28, 0.0208333333),
  (30, 0.021428571400000002),
  (39, 0.0344827586),
  (59, 0.023255814000000003),
  (65, 0.0108303249),
  (67, 0.025),
  (71, 0.0344827586),
  (93, 0.03),
  (101, 0.0158730159),
  (105, 0.0344827586),
  (110, 0.0326797386),
  (134, 0.013245033100000001),
  (135, 0.0173010381),
  (139, 0.0344827586),
  (155, 0.0272727273),
  (164, 0.019736842100000002),
  (169, 0.0098039216),
  (175, 0.0344827586),
  (204, 0.033333333300000004),
  (209, 0.0344827586),
  (245, 0.0344827586),
  (297, 0.027027027000000002),
  (298, 0.06),
  (314, 0.05),
  (403, 0.0428571429),
  (647, 0.0909090909),
  (677, 0.0428571429),
  (686, 0.0101522843),
  (703, 0.1),
  (893, 0.009980039900000001),
  (912, 0.0238500852),
  (925, 0.0075101098),
  (957, 0.033333333300000004),
  (991, 0.009345794400000001)],
 'year': [(0, 0.025),
  (13, 0.025),
  (17, 0.0272727273),
  (38, 0.0104166667),
  (39, 0.025),
  (51, 0.0272727273),


In [73]:
df.head(2)

,title,url,date,transcript,source,keywords
0,16-Year-Old Has Been Using His Flying Lessons ...,https://www.goodnewsnetwork.org/teen-flies-sup...,4/9/20,16-year-old TJ Kim may not have his driver’s l...,goodnewsnetwork.org,"{'hospit': 0.0344827586, 'year': 0.025, 'old':..."
1,Tyler Perry Picks Up the Tab for All Groceries...,https://www.goodnewsnetwork.org/tyler-perry-pa...,4/9/20,It’s not uncommon for movie mogul Tyler Perry ...,goodnewsnetwork.org,"{'perri': 0.0384615385, 'tyler': 0.0320512821,..."


In [92]:
import json

In [93]:
with open('inverted_index_keywords.json', 'w') as f:
        f.write(json.dumps(invidx, ensure_ascii=False,indent=2))

In [103]:
def search_keyword_method(query, n_docs, input_inverted_index, tokenize_method):
    q_tokens = tokenize_method(query)
    scores = np.zeros(n_docs)
    for t in set(q_tokens):
         if (input_inverted_index.get(t) is not None):
            for tup in input_inverted_index.get(t):
                doc_id = tup[0]
                scores[doc_id] += tup[1]
    sim = []
    for i in range(n_docs):
        if(scores[i] > 0):
            sim.append((scores[i], i))
    sim.sort(key = lambda x: x[0], reverse = True)
    return sim

In [101]:
import numpy as np

In [99]:
query = "pizza"

In [108]:
results = search_keyword_method(query, 1062, invidx, tokenize2)

In [113]:
top_5 = get_top_k(results, 5, total_data_2)

In [114]:
top_5

[('Outback roadhouse flies beer and pizza to cattle stations in lockdown',
  'https://thenewdaily.com.au/news/coronavirus/2020/04/03/coronavirus-outback-pizza/'),
 ('18-Year-Old Drove 225 Miles to Make Pizzeria’s First and Only Delivery After Receiving Special Phone Call',
  'https://www.goodnewsnetwork.org/teen-makes-pizzerias-one-and-only-delivery-for-a-hospice-patient-225-miles-away/'),
 ('Businesses Are Giving Away All Their Excess Toilet Paper to People in Need During COVID-19 Shutdowns',
  'https://www.goodnewsnetwork.org/businesses-giving-away-toilet-paper-during-covid-shutdowns/')]

In [112]:
with open('total_data_2_keywords.json', encoding = 'utf-8') as json_file:
        total_data_2 = json.load(json_file)

In [106]:
def get_top_k(results, k, input_data):
    output = []
    for i in range(min(k, len(results))):
        doc_id = results[i][1]
        title = (input_data[doc_id]['title'], input_data[doc_id].get('url'))
        output.append(title)
    return output

In [97]:
def tokenize2(text, stem_words = True):
    """Returns a list of words that make up the text.
    
    Note: for simplicity, lowercase everything.
    Requirement: Use Regex to satisfy this function
    
    Arguments
    =========
    text: String
    stem_words: boolean
    
    Returns
    =======
    List
    
    """
    stop_words = stopwords.words('english')
    stemmer = PorterStemmer()
    regex = r"([a-zA-Z]+)"
    tokens = re.findall(regex, text.lower())
    if (stem_words):
        return [stemmer.stem(t) for t in tokens if not t in stop_words]
    else:
        return [t for t in tokens if not t in stop_words]